In [7]:
from __future__ import division
from pyspark import SparkConf, SparkContext
from operator import add
import re
import glob, os
import csv
from collections import Counter
import ntpath
import functools
from functools import reduce
from math import log
APP_NAME="NFP_PRS"


# PRS calculation on pruned NFP data, 50, 5, VIF=2

#**ATTN: python index starts at 0, so if you want to specify the second column, use 1
#**ATTN: please remove the header of the GWAS file if there is any

# define column number for contents in GWAS

gwas_id=0    # column of SNP ID
gwas_p=7     # column of P value
gwas_or=5    # column of odds ratio
gwas_a1=3    # column of a1 in the GWAS

# defin column number for contents in genfile
geno_id=1  # column number with rsID
geno_start=5 # column number of the 1st genotype
geno_a1 = 3  # column number that contains the reference allele

# List of thresholds:
thresholds=[0.5, 0.3, 0.2, 0.1, 0.05, 0.01, 0.001, 0.0001]

# file delimiters:
GWAS_delim="\t"
GENO_delim=" "

# file names:
home="/home/nyao111/"  #define homefolder path
gwasFiles=home+"PRS_imputed/pgc.cross.noheader.txt"       # Name of GWAS file 


def getFileFromPattern(*pattern):
    files=[]
    for pathpattern in pattern:
        files=glob.glob(files)

genoFileNamePattern=home+"NFPimputed_pruned.gen"   # Multiple patterns need to be put into list format

genoFileNames=glob.glob(genoFileNamePattern)
# Alternatively, directly specify filename:
#genoFileName=[home+"fcgene_out_chr21comb.bierut1M_plus_filtered_chr21_c1_EA_COGA.gen",
              #home+"fcgene_out_chr21comb.bierut1M_plus_filtered_chr21_c1_EA_COGEND.gen",
              #home+"fcgene_out_chr22comb.bierut1M_plus_filtered_chr22_c1_EA_COGA.gen",
              #home+"fcgene_out_chr22comb.bierut1M_plus_filtered_chr22_c1_EA_COGEND.gen"]

genoExtension=".gen"


# programme parameters
log_or=True  # sepcify whether you want to log your odds ratios
check_ref=True # if you know that there are mismatch between the top strand in the genotypes and that of the GWAS, set True. Not checking the reference allele will improve the speed

# sample file path and name
sampleFilePath="NFPimputed_pruned.sample" # include the full/relative path and name of the sample file
sampleFileDelim=" "  # sample File Delimiter
sampleFileID=0   # which column in the sample file has the ID
sample_skip=2  # how many lines to skip so that the sample names can be matched to the genotypes 1-to-1
##output file information

outputPath=home+"NFP_prs.csv"



In [34]:
%autoreload 2
from PRSfunctions import calcPRSFromGeno


ImportError: cannot import name 'calcPRSFromGeno'

In [5]:
import pyspark
from pyspark.sql import SQLContext

# We can give a name to our app (to find it in Spark WebUI) and configure execution mode
# In this case, it is local multicore execution with "local[*]"


app_name = "PRS"
master = "local"
conf = pyspark.SparkConf().setAppName(app_name).setMaster(master)
sc = pyspark.SparkContext(conf=conf)
print(sc)

In [6]:
sc.setLogLevel("WARN")
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
LOGGER.info("Start Reading Files")
#def main(gwasFile, genoFileList, thresholdList):
print("="*40)
print("Using these genoytpe files: ")

counter = 0
for filename in genoFileNames:
    if counter<20:
        counter+=1
        print(filename)
    else:
        print("and more....")
        break


genodata=sc.textFile(genoFileNamePattern)
genotable=genodata.map(lambda line: line.split(GENO_delim))
gwasfile=sc.textFile(gwasFiles)
print("Using the GWAS file: ")
print(ntpath.basename(gwasFiles))

gwastable=gwasfile.map(lambda line: line.split(GWAS_delim))

gwastableCA=gwastable.cache()
# Find the maximum threshold
# Filter the genotypes to keep only SNPs with p-values less than this threshold
# And then cache the filtered genotypes


maxThreshold=max(thresholds)

Using these genoytpe files: 
/home/nyao111/NFPimputed_pruned.gen
Using the GWAS file: 
pgc.cross.noheader.txt


In [30]:
#import pickle
#NFP_maf=genotable.map(lambda line: getMaf(line))
#MAFmap=NFP_maf.collectAsMap()
#with open('/scratch/vvp-220-aa/NFP_allele_frequency', 'wb') as handle:
    #pickle.dump(MAFmap, handle)

In [8]:
gwasFilteredMax=filterGWASByP(GWASRdd=gwastableCA, pcolumn=gwas_p, idcolumn=gwas_id, oddscolumn=gwas_or, pHigh=maxThreshold, logOdds=log_or)
gwasFilteredMaxCA=sc.broadcast(gwasFilteredMax)

Taking the log of odds-ratios


In [9]:
# filter the raw genotype file

genoFilteredMax=genotable.filter(lambda line: line[geno_id] in gwasFilteredMax)

# make a dictionary for the top strand for each SNP
#topStrands=gwastableCA.map(lambda line: (line[gwas_id],line[gwas_a1])).collectAsMap()
#topStrandsBC=sc.broadcast(topStrands)

if check_ref:
    alleleMap=gwasAllelesMap(gwastable)
    alleleMapBC=sc.broadcast(alleleMap)
    genotypeMaxPre=genoFilteredMax.map(lambda line: makeGenotypeCheckRef(line, strandMap=alleleMapBC.value, bpMap=bpPair))
    genotypeMax=genotypeMaxPre.filter(lambda line: line is not None)
    genotypeMaxCA=genotypeMax.cache()
else:
    genotypeMax=genoFilteredMax.map(lambda line: makeGenotype(line, gwasFilteredMaxCA.value))
    genotypeMaxCA=genotypeMax.cache()

In [16]:
len(genotypeMaxCA.first()[1])

192

In [10]:
genoFilteredMax.count()

164958

In [11]:
genoAmbi=genoFilteredMax.filter(lambda line: line[geno_a1]==bpPair[line[geno_a1+1]])
genoAmbi.count()

12682

In [13]:
genoNormal=genoFilteredMax.filter(lambda line: line[geno_a1]!=bpPair[line[geno_a1+1]])

In [22]:
genoNormal.map(lambda line: "\t".join(line)).saveAsTextFile("NFP_genfile_p0_5_noambi_plain")

In [ ]:
# Calculate the PRS with the maximum threshold
prsMax=calcPRSFromGeno(genotypeMaxCA, gwasFilteredMax)
prsDict={}
prsDict[maxThreshold]=prsMax

# Calculate PRS for the rest of the thresholds

In [ ]:
if len(thresholds)>1:
    thresholdListNoMax=[x for x in thresholds if x != maxThreshold]

for threshold in thresholdListNoMax:
    gwasFiltered=filterGWASByP(GWASRdd=gwastableCA, pcolumn=gwas_p, idcolumn=gwas_id, oddscolumn=gwas_or, pHigh=threshold, logOdds=log_or)
    gwasFilteredBC=sc.broadcast(gwasFiltered)
    genoTypeFiltered=genotypeMaxCA.filter(lambda line: line[0] in gwasFilteredBC.value)
    prsOther=calcPRSFromGeno(genoTypeFiltered, gwasFiltered)
    prsDict[threshold]=prsOther
    print("finished calculating PRS at threshold of "+str(threshold))
## putting labels on scores and write scores to file

In [ ]:
subjNames=getSampleNames(scores=prsMax)
pvalues, scores=labelPRS(prsDict, subjNames)
writePRS(prsTable=scores, outputFile=outputPath, pvalues=pvalues)

In [ ]:
outputPath